In [ ]:
from neo4j.v1 import GraphDatabase
import pandas as pd
import json

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "zelda1"))

In [ ]:
def escape(string):
    return json.dumps(string)[1:-1]

## Entities

In [ ]:
entities = pd.read_csv("../relation_extraction/info/entities.csv", index_col=0)
entities.head(2)

In [ ]:
create_template = "CREATE (e:Entity{name:\"%s\"})"
with driver.session() as session:
    for i, row in entities.iterrows():
        name = escape(row['name'])
        insert_stmt = create_template % (name) 
        result = session.run(insert_stmt)
    session.run("CREATE INDEX ON :Entity(name)")

## Resources

In [ ]:
resources = pd.read_csv("../relation_extraction/info/urls.csv", index_col=0)
resources.head(2)

In [ ]:
create_template = "CREATE (e:Resource{uri:\"%s\"})"
with driver.session() as session:
    for i, row in resources.iterrows():
        url = row['url']
        insert_stmt = create_template % (url) 
        result = session.run(insert_stmt)
    session.run("CREATE INDEX ON :Resource(uri)")

# `Represents` creation

In [ ]:
wikia_rels = pd.read_csv("../relation_extraction/info/entities.wikia.csv", index_col=0)
gamepedia_rels = pd.read_csv("../relation_extraction/info/entities.gamepedia.csv", index_col=0)

In [ ]:
relationship_template = """MATCH (from:Resource{uri:\"%s\"}),(to:Entity{name:\"%s\"})
MERGE (from)-[r:Represents]->(to)
SET r.%s=true"""

for site, rels in zip(["wikia","gamepedia"],[ wikia_rels, gamepedia_rels]):
    print(site)
    with driver.session() as session:
        for i, row in rels.iterrows():
            if (i+1 )% 1500 == 0:
                print(i)
            try:
                name = escape(row['name'])
                relationship_stmt = relationship_template % (row['page'], name, site)
                session.run(relationship_stmt)
            except Exception as inst:
                print("Error", i)
                print(inst)
                break

## Other relationships creation

In [ ]:
hard_relationships = pd.read_csv("../relation_extraction/info/hard_relationships.csv", index_col=0)
hard_relationships.head(2)

In [ ]:
relationship_template = """MATCH (from:Resource{uri:\"%s\"}),(to:Resource{uri:\"%s\"})
MERGE (from)-[r:%s{verb:\"%s\",preposition:\"%s\"}]->(to)"""

with driver.session() as session:
    for i, r in hard_relationships.iterrows():
        one = r["url"]
        verb = r["relation1"]
        preposition = r["relation2"]
        relation = r["attribute"]
        two = r["related_url"]
        create_stmt = relationship_template % (one, two, relation, verb, preposition)
        session.run(create_stmt)